# Importing libraries

In [2]:
import re
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shengjie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shengjie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Importing data

In [32]:
d1_path = "./data/processed_tsv/K1.tsv"
d2_path = "./data/processed_tsv/K2.tsv"
d3_path = "./data/processed_tsv/K3.tsv"
d4_path = "./data/processed_tsv/LIAR.tsv"

d1 = pd.read_csv(d1_path, delimiter="\t")
d2 = pd.read_csv(d2_path, delimiter="\t")
d3 = pd.read_csv(d3_path, delimiter="\t")
d4 = pd.read_csv(d4_path, delimiter="\t")

d1['content'] = d1['content'].apply(lambda x: str(x))
d2['content'] = d2['content'].apply(lambda x: str(x))
d3['content'] = d3['content'].apply(lambda x: str(x))
d4['content'] = d4['content'].apply(lambda x: str(x))

d1['domain'] = 0
d2['domain'] = 1
d3['domain'] = 2
d4['domain'] = 3


In [33]:
print('===== d1 =====\n', d1.groupby(['label'])[['label']].count())
print('===== d2 =====\n', d2.groupby(['label'])[['label']].count())
print('===== d3 =====\n', d3.groupby(['label'])[['label']].count())
print('===== d4 =====\n', d4.groupby(['label'])[['label']].count())


===== d1 =====
        label
label       
0      23481
1      21417
===== d2 =====
        label
label       
0       4488
1       5752
===== d3 =====
        label
label       
0       2137
1       1872
===== d4 =====
        label
label       
0       3554
1       4507


# Text preprocessing

In [34]:
class Preprocess:

    def __init__(self):
        self.wordnet_lemmatizer = WordNetLemmatizer()
        self.porter = PorterStemmer()

    def clean_text(self, text):
        '''Clean text by removing unnecessary characters and altering the format of words.'''

        text = str(text)
        text = text.lower()
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"what's", "that is", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"how's", "how is", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"n't", " not", text)
        text = re.sub(r"n'", "ng", text)
        text = re.sub(r"'bout", "about", text)
        text = re.sub(r"'til", "until", text)
        text = re.sub(r"[()\"_#/@;*%:{}<>`+=~|.!?,'$-\[\]]", "", text)
        text = re.sub(r"[0-9]", "", text)

        return text

    def lemmatizer(self, text):
        sentence_words = nltk.word_tokenize(text)
        ret_text = []
        for word in sentence_words:
            ret_text.append(self.wordnet_lemmatizer.lemmatize(word))

        return " ".join(ret_text)

    def stemmer(self, text):
        sentence_words = nltk.word_tokenize(text)
        ret_text = []
        for word in sentence_words:
            ret_text.append(self.porter.stem(word))

        return " ".join(ret_text)


# Formating data

In [35]:
def encoder_data(df, lemmatize=False, stem=False):
    preprocessor = Preprocess()
    encoder_inputs = df['content'].apply(lambda x: preprocessor.clean_text(x))
    if lemmatize:
        encoder_inputs = df['content'].apply(
            lambda x: preprocessor.lemmatizer(x))
    if stem:
        encoder_inputs = df['content'].apply(lambda x: preprocessor.stemmer(x))
    encoder_inputs = np.array(encoder_inputs.values.tolist())
    return encoder_inputs


def fake_news_target(df):
    return np.array(df['label'].apply(lambda x: int(x)).values.tolist())


def domain_target(df):
    return np.array(df['domain'].apply(lambda x: int(x)).values.tolist())


class Shuffle:

    def __init__(self):
        self.seed = np.random.randint(100000)

    def shuffle(self, data):
        np.random.seed(self.seed)
        np.random.shuffle(data)
        return data


In [36]:
# Creating the word2idx, idx2word mapping using the Keras Tokenizer
def create_vocab(text_lists, vocab_len):
    tokenizer = Tokenizer(oov_token="<UNK>")
    tokenizer.fit_on_texts(text_lists)
    # Due to ambiguity with regards to Keras Tokenizer num_words, below is a good enough fix,
    # though it changes the tokenizer word_index outside of the class
    num_words = vocab_len

    sorted_by_word_count = sorted(
        tokenizer.word_counts.items(), key=lambda kv: kv[1], reverse=True)
    tokenizer.word_index = {}
    word2idx = {}
    idx2word = {}
    i = 0
    for word, count in sorted_by_word_count:
        if i == num_words:
            break

        # <= because tokenizer is 1 indexed
        tokenizer.word_index[word] = i + 1
        word2idx[word] = i+1
        idx2word[i+1] = word
        i += 1

    tokenizer.word_index[tokenizer.oov_token] = num_words+1
    word2idx[tokenizer.oov_token] = num_words+1
    idx2word[num_words+1] = tokenizer.oov_token

    return word2idx, idx2word, tokenizer


def pad_tokenize_data(encoder_inputs, max_sentence_length, tokenizer):

    t_encoder_inputs = tokenizer.texts_to_sequences(encoder_inputs)
    t_encoder_inputs = pad_sequences(
        t_encoder_inputs, maxlen=max_sentence_length, padding='post', truncating='post')

    return t_encoder_inputs


In [37]:
def batch_generator(X, Y_CC, Y_DC, max_sentence_length, word2idx, batch_size=128):

    y_cc = to_categorical(Y_CC)
    y_dc = to_categorical(Y_DC)
    for idx in range(0, len(X), batch_size):
        encoder_input = np.zeros((batch_size, max_sentence_length))
        decoder_target = np.zeros(
            (batch_size, max_sentence_length, len(word2idx)+1))
        for j, input_seq in enumerate(X[idx:idx+batch_size]):
            for i, word_idx in enumerate(input_seq):
                encoder_input[j, i] = word_idx
                decoder_target[j, i, word_idx] = 1

        yield [encoder_input, [decoder_target, y_cc[idx:idx+batch_size], y_dc[idx:idx+batch_size]]]


def all_data_generator(X, Y_CC, Y_DC, max_sentence_length):
    encoder_input = np.zeros((len(X), max_sentence_length))
    for j, input_seq in enumerate(X):
        for i, word_idx in enumerate(input_seq):
            encoder_input[j, i] = word_idx

    y_cc = to_categorical(Y_CC)
    y_dc = to_categorical(Y_DC)

    return [encoder_input, y_cc, y_dc]


# Building models

## Model for a single domain

In [38]:
def classification_model(max_encoder_len, embedding_dim, latent_dim, vocab_len):
    inputs = Input(shape=(max_encoder_len,), name="encoder_inputs")

    layer_embedding = Embedding(vocab_len+1, embedding_dim, trainable=True,
                                input_length=max_encoder_len, mask_zero=True, name="encoder_embedding")
    layer_lstm = LSTM(latent_dim, return_state=True)
    layer_nonlinear = Dense(128, activation="tanh", name="non_linear")
    layer_softmax = Dense(2, activation="softmax", name="softmax")

    embedded = layer_embedding(inputs)
    encoded, _, _ = layer_lstm(embedded)
    logits = layer_nonlinear(encoded)
    outputs = layer_softmax(logits)

    return Model(inputs, outputs)


## Domain independent model

In [51]:
def domain_independent_model(max_encoder_len, embedding_dim, latent_dim, vocab_len, domain_count):
    inputs = Input(shape=(max_encoder_len,), name="encoder_inputs")

    layer_embedding = Embedding(vocab_len+1, embedding_dim, trainable=True,
                                input_length=max_encoder_len, mask_zero=True, name="encoder_embedding")
    layer_lstm = LSTM(latent_dim, return_state=True)
    layer_nonlinear_cc = Dense(128, activation="tanh", name="nonlinear_cc")
    layer_softmax_cc = Dense(2, activation="softmax", name="softmax_cc")
    layer_nonlinear_dc = Dense(128, activation="tanh", name="nonlinear_dc")
    layer_softmax_dc = Dense(domain_count, activation="softmax", name="softmax_dc")

    embedded = layer_embedding(inputs)
    encoded, _, _ = layer_lstm(embedded)
    logits_cc = layer_nonlinear_cc(encoded)
    outputs_cc = layer_softmax_cc(logits_cc)
    logits_dc = layer_nonlinear_dc(encoded)
    outputs_dc = layer_softmax_dc(logits_dc)

    return Model(inputs, [outputs_cc, outputs_dc])


# Experiments

## Experiment 1

In [57]:
def prepare_training_data(df, vocab_size, max_sentence_length):
    encoder_inputs = encoder_data(df)
    y_cc = fake_news_target(df)
    y_dc = domain_target(df)

    # Initializing the shuffle class instance
    shuffle = Shuffle()
    encoder_inputs = shuffle.shuffle(encoder_inputs)
    y_cc = shuffle.shuffle(y_cc)
    y_dc = shuffle.shuffle(y_dc)  # Not used

    word2idx, idx2word, tokenizer = create_vocab(encoder_inputs, vocab_size)
    vocab_len = len(word2idx)

    encoder_inputs = pad_tokenize_data(
        encoder_inputs, max_sentence_length, tokenizer)

    encoder_inputs, y_cc, y_dc = all_data_generator(
        encoder_inputs, y_cc, y_dc, max_sentence_length)

    train_X, test_X, train_Y, test_Y = train_test_split(
        encoder_inputs, y_cc, test_size=0.4, random_state=42)

    return [[train_X, test_X, train_Y, test_Y], [word2idx, idx2word, tokenizer], vocab_len]


def prepare_testing_data(df, max_sentence_length):
    encoder_inputs = encoder_data(df)
    y_cc = fake_news_target(df)
    y_dc = domain_target(df)

    encoder_inputs = pad_tokenize_data(
        encoder_inputs, max_sentence_length, tokenizer)

    encoder_inputs, y_cc, y_dc = all_data_generator(
        encoder_inputs, y_cc, y_dc, max_sentence_length)

    return [encoder_inputs, y_cc, y_dc]


In [41]:
max_sentence_length = 50
embedding_dim = 100
latent_dim = 64
vocab_size = 500

[train_X, test_X, train_Y, test_Y], \
    [word2idx, idx2word, tokenizer], vocab_len = prepare_training_data(
        d1, vocab_size, max_sentence_length)


In [29]:
cc_model = classification_model(
    max_sentence_length, embedding_dim, latent_dim, vocab_len)
cc_model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_inputs (InputLayer)  [(None, 50)]             0         
                                                                 
 encoder_embedding (Embeddin  (None, 50, 100)          50200     
 g)                                                              
                                                                 
 lstm_3 (LSTM)               [(None, 64),              42240     
                              (None, 64),                        
                              (None, 64)]                        
                                                                 
 non_linear (Dense)          (None, 128)               8320      
                                                                 
 softmax (Dense)             (None, 2)                 258       
                                                           

In [30]:
cc_model.compile(optimizer="rmsprop",
                 loss='binary_crossentropy', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1)
mcp_save = ModelCheckpoint(
    '.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
history = cc_model.fit(train_X,
                       train_Y,
                       batch_size=256,
                       validation_split=0.1,
                       callbacks=[es, mcp_save, reduce_lr_loss],
                       epochs=30)


Epoch 1/30
95/95 [==============================] - 12s 99ms/step - loss: 0.3099 - accuracy: 0.8657 - val_loss: 0.2050 - val_accuracy: 0.9258 - lr: 0.0010
Epoch 2/30
95/95 [==============================] - 8s 88ms/step - loss: 0.1894 - accuracy: 0.9331 - val_loss: 0.1839 - val_accuracy: 0.9365 - lr: 0.0010
Epoch 3/30
95/95 [==============================] - 9s 91ms/step - loss: 0.1742 - accuracy: 0.9394 - val_loss: 0.1869 - val_accuracy: 0.9313 - lr: 0.0010
Epoch 4/30
95/95 [==============================] - 9s 92ms/step - loss: 0.1652 - accuracy: 0.9412 - val_loss: 0.1845 - val_accuracy: 0.9358 - lr: 0.0010
Epoch 5/30
95/95 [==============================] - 9s 90ms/step - loss: 0.1551 - accuracy: 0.9434 - val_loss: 0.1747 - val_accuracy: 0.9395 - lr: 0.0010
Epoch 6/30
95/95 [==============================] - 9s 92ms/step - loss: 0.1486 - accuracy: 0.9470 - val_loss: 0.2077 - val_accuracy: 0.9362 - lr: 0.0010
Epoch 7/30
95/95 [==============================] - 9s 91ms/step - loss: 0.

In [43]:
y_pred = cc_model.predict(test_X)
y_pred = np.array([np.argmax(x) for x in y_pred])
y_true = np.array([np.argmax(x) for x in test_Y])

print(f"Prediction acuracy on same domain is {round(accuracy_score(y_true, y_pred), 3)}")
print(f"Precision on same domain is {round(precision_score(y_true, y_pred), 3)}")
print(f"Recall on same domain is {round(recall_score(y_true, y_pred), 3)}")
print(f"F1 on same domain is {round(f1_score(y_true, y_pred), 3)}")

cmtx = pd.DataFrame(
    confusion_matrix(y_true, y_pred, labels=[1, 0]),
    index=['true:Fake', 'true:Real'],
    columns=['pred:Fake', 'pred:Real']
)
print(cmtx)


for df in [d2, d3, d4]:
    df_encoder_inputs, df_y_cc, df_y_dc = prepare_testing_data(
        df, max_sentence_length)

    y_pred = cc_model.predict(df_encoder_inputs)
    y_pred = np.array([np.argmax(x) for x in y_pred])
    y_true = np.array([np.argmax(x) for x in df_y_cc])

    print("\n")
    print(f"Prediction acuracy on different domain is {round(accuracy_score(y_true, y_pred), 3)}")
    print(f"Precision on different domain is {round(precision_score(y_true, y_pred), 3)}")
    print(f"Recall on different domain is {round(recall_score(y_true, y_pred), 3)}")
    print(f"F1 on different domain is {round(f1_score(y_true, y_pred), 3)}")
    cmtx = pd.DataFrame(
        confusion_matrix(y_true, y_pred, labels=[1, 0]),
        index=['true:Fake', 'true:Real'],
        columns=['pred:Fake', 'pred:Real']
    )
    print(cmtx)


Prediction acuracy on same domain is 0.947
Precision on same domain is 0.953
Recall on same domain is 0.936
F1 on same domain is 0.944
           pred:Fake  pred:Real
true:Fake       8061        555
true:Real        396       8948


Prediction acuracy on different domain is 0.459
Precision on different domain is 0.632
Recall on different domain is 0.09
F1 on different domain is 0.157
           pred:Fake  pred:Real
true:Fake        515       5237
true:Real        300       4188


Prediction acuracy on different domain is 0.705
Precision on different domain is 0.884
Recall on different domain is 0.425
F1 on different domain is 0.574
           pred:Fake  pred:Real
true:Fake        795       1077
true:Real        104       2033


Prediction acuracy on different domain is 0.465
Precision on different domain is 0.656
Recall on different domain is 0.091
F1 on different domain is 0.16
           pred:Fake  pred:Real
true:Fake        411       4096
true:Real        216       3338


## Experiment 1.5

In [61]:
max_sentence_length = 50
embedding_dim = 100
latent_dim = 64
vocab_size = 500

[train_X, test_X, train_Y, test_Y], \
    [word2idx, idx2word, tokenizer], vocab_len = prepare_training_data(
        pd.concat([d1, d2, d3, d4]), vocab_size, max_sentence_length)


In [62]:

cc_model.compile(optimizer="rmsprop",
                 loss='binary_crossentropy', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1)
mcp_save = ModelCheckpoint(
    '.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
history = cc_model.fit(train_X,
                       train_Y,
                       batch_size=256,
                       validation_split=0.1,
                       callbacks=[es, mcp_save, reduce_lr_loss],
                       epochs=30)


Epoch 1/30
142/142 [==============================] - 24s 147ms/step - loss: 0.5140 - accuracy: 0.7343 - val_loss: 0.3498 - val_accuracy: 0.8284 - lr: 0.0010
Epoch 2/30
142/142 [==============================] - 21s 145ms/step - loss: 0.3459 - accuracy: 0.8291 - val_loss: 0.3289 - val_accuracy: 0.8344 - lr: 0.0010
Epoch 3/30
142/142 [==============================] - 21s 148ms/step - loss: 0.3263 - accuracy: 0.8413 - val_loss: 0.3322 - val_accuracy: 0.8311 - lr: 0.0010
Epoch 4/30
142/142 [==============================] - 21s 148ms/step - loss: 0.3136 - accuracy: 0.8465 - val_loss: 0.3389 - val_accuracy: 0.8354 - lr: 0.0010
Epoch 5/30
142/142 [==============================] - 21s 147ms/step - loss: 0.3048 - accuracy: 0.8521 - val_loss: 0.3314 - val_accuracy: 0.8329 - lr: 0.0010
Epoch 6/30
142/142 [==============================] - 21s 148ms/step - loss: 0.2973 - accuracy: 0.8576 - val_loss: 0.3309 - val_accuracy: 0.8354 - lr: 0.0010
Epoch 7/30
142/142 [==============================] 

In [63]:

y_pred = cc_model.predict(test_X)
y_pred = np.array([np.argmax(x) for x in y_pred])
y_true = np.array([np.argmax(x) for x in test_Y])

print(f"Prediction acuracy on all domain is {round(accuracy_score(y_true, y_pred), 3)}")
print(f"Precision on all domain is {round(precision_score(y_true, y_pred), 3)}")
print(f"Recall on all domain is {round(recall_score(y_true, y_pred), 3)}")
print(f"F1 on all domain is {round(f1_score(y_true, y_pred), 3)}")

cmtx = pd.DataFrame(
    confusion_matrix(y_true, y_pred, labels=[1, 0]),
    index=['true:Fake', 'true:Real'],
    columns=['pred:Fake', 'pred:Real']
)
print(cmtx)


for df in [d1, d2, d3, d4]:
    df_encoder_inputs, df_y_cc, df_y_dc = prepare_testing_data(
        df, max_sentence_length)

    y_pred = cc_model.predict(df_encoder_inputs)
    y_pred = np.array([np.argmax(x) for x in y_pred])
    y_true = np.array([np.argmax(x) for x in df_y_cc])

    print("\n")
    print(f"Prediction acuracy on different domain is {round(accuracy_score(y_true, y_pred), 3)}")
    print(f"Precision on different domain is {round(precision_score(y_true, y_pred), 3)}")
    print(f"Recall on different domain is {round(recall_score(y_true, y_pred), 3)}")
    print(f"F1 on different domain is {round(f1_score(y_true, y_pred), 3)}")
    cmtx = pd.DataFrame(
        confusion_matrix(y_true, y_pred, labels=[1, 0]),
        index=['true:Fake', 'true:Real'],
        columns=['pred:Fake', 'pred:Real']
    )
    print(cmtx)


Prediction acuracy on all domain is 0.831
Precision on all domain is 0.859
Recall on all domain is 0.79
F1 on all domain is 0.823
           pred:Fake  pred:Real
true:Fake      10584       2817
true:Real       1738      11745


Prediction acuracy on different domain is 0.939
Precision on different domain is 0.961
Recall on different domain is 0.91
F1 on different domain is 0.935
           pred:Fake  pred:Real
true:Fake      19490       1927
true:Real        797      22684


Prediction acuracy on different domain is 0.621
Precision on different domain is 0.676
Recall on different domain is 0.626
F1 on different domain is 0.65
           pred:Fake  pred:Real
true:Fake       3599       2153
true:Real       1726       2762


Prediction acuracy on different domain is 0.807
Precision on different domain is 0.972
Recall on different domain is 0.604
F1 on different domain is 0.745
           pred:Fake  pred:Real
true:Fake       1130        742
true:Real         33       2104


Prediction acur

## Experiment 2

In [44]:
def prepare_training_data(df, vocab_size, max_sentence_length):
    encoder_inputs = encoder_data(df)
    y_cc = fake_news_target(df)
    y_dc = domain_target(df)

    # Initializing the shuffle class instance
    shuffle = Shuffle()
    encoder_inputs = shuffle.shuffle(encoder_inputs)
    y_cc = shuffle.shuffle(y_cc)
    y_dc = shuffle.shuffle(y_dc)

    word2idx, idx2word, tokenizer = create_vocab(encoder_inputs, vocab_size)
    vocab_len = len(word2idx)

    encoder_inputs = pad_tokenize_data(
        encoder_inputs, max_sentence_length, tokenizer)

    encoder_inputs, y_cc, y_dc = all_data_generator(
        encoder_inputs, y_cc, y_dc, max_sentence_length)

    train_X, test_X, train_C_Y, test_C_Y, train_D_Y, test_D_Y = train_test_split(
        encoder_inputs, y_cc, y_dc, test_size=0.4, random_state=42)

    return [[train_X, test_X, train_C_Y, test_C_Y, train_D_Y, test_D_Y], [word2idx, idx2word, tokenizer], vocab_len]

def prepare_testing_data(df, max_sentence_length):
    encoder_inputs = encoder_data(df)
    y_cc = fake_news_target(df)
    y_dc = domain_target(df)

    encoder_inputs = pad_tokenize_data(
        encoder_inputs, max_sentence_length, tokenizer)

    encoder_inputs, y_cc, y_dc = all_data_generator(
        encoder_inputs, y_cc, y_dc, max_sentence_length)

    return [encoder_inputs, y_cc, y_dc]


In [45]:
max_sentence_length = 100
embedding_dim = 100
latent_dim = 64
vocab_size = 500

[train_X, test_X, train_C_Y, test_C_Y, train_D_Y, test_D_Y], \
    [word2idx, idx2word, tokenizer], vocab_len = prepare_training_data(
        pd.concat([d1, d2, d3, d4]), vocab_size, max_sentence_length)

cc_model = classification_model(
    max_sentence_length, embedding_dim, latent_dim, vocab_len)
cc_model.summary()

In [52]:

dc_model = domain_independent_model(
    max_sentence_length, embedding_dim, latent_dim, vocab_len, domain_count=4)
dc_model.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 100, 100)     50200       ['encoder_inputs[0][0]']         
                                                                                                  
 lstm_5 (LSTM)                  [(None, 64),         42240       ['encoder_embedding[0][0]']      
                                 (None, 64),                                                      
                                 (None, 64)]                                                      
                                                                                            

In [53]:
dc_model.compile(optimizer="rmsprop", loss=[
    'binary_crossentropy', 'binary_crossentropy'], loss_weights=[0.7, -0.2], metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', patience=30, verbose=1)
mcp_save = ModelCheckpoint(
    '.md2_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
history = dc_model.fit(train_X,
                       [train_C_Y, train_D_Y],
                       batch_size=256,
                       validation_split=0.1,
                       callbacks=[es, mcp_save, reduce_lr_loss],
                       epochs=30)


Epoch 1/30
142/142 [==============================] - 37s 237ms/step - loss: -2.2183 - softmax_cc_loss: 0.6972 - softmax_dc_loss: 13.5317 - softmax_cc_accuracy: 0.5163 - softmax_dc_accuracy: 0.0617 - val_loss: -3.9013 - val_softmax_cc_loss: 0.6697 - val_softmax_dc_loss: 21.8504 - val_softmax_cc_accuracy: 0.5986 - val_softmax_dc_accuracy: 0.0565 - lr: 0.0010
Epoch 2/30
142/142 [==============================] - 31s 216ms/step - loss: -5.4494 - softmax_cc_loss: 0.6125 - softmax_dc_loss: 29.3908 - softmax_cc_accuracy: 0.6796 - softmax_dc_accuracy: 0.0608 - val_loss: -6.9182 - val_softmax_cc_loss: 0.7273 - val_softmax_dc_loss: 37.1366 - val_softmax_cc_accuracy: 0.6561 - val_softmax_dc_accuracy: 0.0565 - lr: 0.0010
Epoch 3/30
142/142 [==============================] - 30s 213ms/step - loss: -8.5711 - softmax_cc_loss: 0.5160 - softmax_dc_loss: 44.6616 - softmax_cc_accuracy: 0.7492 - softmax_dc_accuracy: 0.0608 - val_loss: -10.1731 - val_softmax_cc_loss: 0.4424 - val_softmax_dc_loss: 52.4138 

In [56]:
y_pred, _ = dc_model.predict(test_X)
y_pred = np.array([np.argmax(x) for x in y_pred])
y_true = np.array([np.argmax(x) for x in test_C_Y])

print(f"Prediction acuracy on all domains is {round(accuracy_score(y_true, y_pred),2)}")
print(f"Precision on all domains is {round(precision_score(y_true, y_pred),2)}")
print(f"Recall on all domains is {round(recall_score(y_true, y_pred),2)}")
print(f"F1 on all domains is {round(f1_score(y_true, y_pred),2)}")
cmtx = pd.DataFrame(
    confusion_matrix(y_true, y_pred, labels=[1, 0]),
    index=['true:Fake', 'true:Real'],
    columns=['pred:Fake', 'pred:Real']
)
print(cmtx)

for df in [d1, d2, d3, d4]:
    df_encoder_inputs, df_y_cc, df_y_dc = prepare_testing_data(
        df, max_sentence_length)

    y_pred, _ = dc_model.predict(df_encoder_inputs)
    y_pred = np.array([np.argmax(x) for x in y_pred])
    y_true = np.array([np.argmax(x) for x in df_y_cc])

    print("\n")
    print(f"Prediction acuracy on different domain is {round(accuracy_score(y_true, y_pred), 3)}")
    print(f"Precision on different domain is {round(precision_score(y_true, y_pred), 3)}")
    print(f"Recall on different domain is {round(recall_score(y_true, y_pred), 3)}")
    print(f"F1 on different domain is {round(f1_score(y_true, y_pred), 3)}")
    cmtx = pd.DataFrame(
        confusion_matrix(y_true, y_pred, labels=[1, 0]),
        index=['true:Fake', 'true:Real'],
        columns=['pred:Fake', 'pred:Real']
    )
    print(cmtx)


Prediction acuracy on all domains is 0.85
Precision on all domains is 0.87
Recall on all domains is 0.83
F1 on all domains is 0.85
           pred:Fake  pred:Real
true:Fake      11156       2232
true:Real       1692      11804


Prediction acuracy on different domain is 0.966
Precision on different domain is 0.966
Recall on different domain is 0.963
F1 on different domain is 0.964
           pred:Fake  pred:Real
true:Fake      20618        799
true:Real        721      22760


Prediction acuracy on different domain is 0.661
Precision on different domain is 0.714
Recall on different domain is 0.66
F1 on different domain is 0.686
           pred:Fake  pred:Real
true:Fake       3796       1956
true:Real       1517       2971


Prediction acuracy on different domain is 0.868
Precision on different domain is 0.962
Recall on different domain is 0.746
F1 on different domain is 0.841
           pred:Fake  pred:Real
true:Fake       1397        475
true:Real         55       2082


Prediction ac